In [61]:
from IPython import get_ipython

if 'google.colab' in str(get_ipython()):
    import torch
    from IPython.core.display import HTML
    from IPython.display import display
    import os
    print("Running in Google Colab")
    if not torch.cuda.is_available():
        display(HTML("""<div style="background-color: red; font-weight: bold; color: white;">You have not activated a GPU in Google Colab. Follow the instructions in the <code style="color: white;">README</code></div>"""))
    print("Installing requirements")
    requirements_url = "https://raw.githubusercontent.com/willdalh/ml-course/main/requirements.txt"
    if not os.path.exists('requirements.txt'):
        !wget {requirements_url}
    %pip install --user -r requirements.txt

## PyTorch

PyTorch er et av flere biblioteker som brukes til trening av dype nevrale nettverk. Et annet kjent alternativ, er TensorFlow. Hva man velger avhenger av hva man er vant med, men begge har sine fordeler og ulemper. I noen tilfeller er PyTorch foretrukket fordi det er mer tydelig hvordan treningsprosessen av nevrale nettverk foregår. PyTorch er også mer brukt i akademia.

In [62]:
import torch

## Vektorer og matriser - generalisert som tensorer
Fra matematikken vet vi at en liste med feks. 3 tall definerer en vektor/punkt som lever i et tre-dimensjonalt rom. I PyTorch representeres dette gjennom datatypen `Tensor`. 

In [63]:
data = [3, 2, 1] # Normal python list
vector = torch.Tensor(data) # Creating a tensor from it
vector

tensor([3., 2., 1.])

Med tensorer er det flere operasjoner som kan gjøres (som vi også til en grad kan forvente av native Python-lister).

In [64]:
print(f"Max value: {vector.max()}")
print(f"Min value: {vector.min()}")

print(f"Index of max element: {vector.argmax()}")
print(f"Index of min element: {vector.argmin()}")

print(f"Sum: {vector.sum()}")

print(f"Sorted:  {torch.sort(vector).values}")

Max value: 3.0
Min value: 1.0
Index of max element: 0
Index of min element: 2
Sum: 6.0
Sorted:  tensor([1., 2., 3.])


Vi kan lage et punkt som lever i flere dimensjoner, for eksempel 16 (som vi heller lager med tilfeldige tall):

In [65]:
high_dim_vector = torch.randint(low=1, high=10, size=(16,))
high_dim_vector

tensor([1, 5, 3, 5, 4, 5, 5, 9, 2, 6, 9, 5, 3, 1, 2, 2])

En matrise er en rektangulær liste / tabell. Det er her `torch.Tensor` begynner å divergere fra vanlige Python-lister.  
Vi lager en $2 \times 3$-matrise, altså med 2 rader og 3 kolonner.

In [66]:
matrix = torch.randint(low=1, high=10, size=(2, 3))
matrix

tensor([[4, 4, 8],
        [8, 9, 8]])

Størrelsen og formen på matrisen kaller vi for _shape_. Er vi usikker på hvilken shape matrisen har, kan vi sjekke det med `.shape` attributtet.

In [67]:
print(f"The shape of the matrix is {matrix.shape}")
rows, cols = matrix.shape
print(f"It has {rows} rows and {cols} columns")

The shape of the matrix is torch.Size([2, 3])
It has 2 rows and 3 columns


Vi kan ta det et steg videre ved å legge til dybde på en matrise.

In [68]:
tensor = torch.randint(low=0, high=10, size=(2, 3, 4))
print(f"The shape of the tensor is {tensor.shape}")
tensor

The shape of the tensor is torch.Size([2, 3, 4])


tensor([[[9, 6, 2, 0],
         [6, 2, 7, 9],
         [7, 3, 3, 4]],

        [[3, 7, 0, 9],
         [0, 9, 6, 9],
         [5, 4, 8, 8]]])

Tensoren over består av 2 matriser som hver har 3 rader og 4 kolonner. Disse kan vi hente ut med indeksering.

In [69]:
matrix_1 = tensor[0]
matrix_2 = tensor[1]
print(f"Matrix 1:\n {matrix_1}")
print(f"Matrix 2:\n {matrix_2}")

Matrix 1:
 tensor([[9, 6, 2, 0],
        [6, 2, 7, 9],
        [7, 3, 3, 4]])
Matrix 2:
 tensor([[3, 7, 0, 9],
        [0, 9, 6, 9],
        [5, 4, 8, 8]])


I praksis brukes ordene _vektor_ og _tensor_ om hverandre. Begge beskriver en samling verdier strukturert med en vilkårlig _shape_. I bunn og grunn er de flerdimensjonale lister/arrays. 

Hva man legger i begrepet _dimensjoner_ når man refererer til en tensor, er todelt:
- Antall elementer til sammen i tensoren (riktig matematisk sett)
- Antall elementer som ligger i `.shape`-attributtet 

For å minske forvirringen kaller man ofte den første for antall _features_. Den andre kan vi kalle for shape-dimensjoner. 

Da gjelder følgende for tensoren vår:

In [70]:
num_features = tensor.numel()
print(f"The tensor has {num_features} features")
num_shape_dims = tensor.dim()
print(f"The tensor has {num_shape_dims} shape dimensions")

The tensor has 24 features
The tensor has 3 shape dimensions


### Funksjonalitet utover det man finner i vanlige Python-lister  
Tensorer er veldig fleksible. Vi kan for eksempel slå sammen radene og kolonnene ved å utføre en `reshape`.

In [71]:
tensor = torch.randint(low=0, high=10, size=(2, 3, 4))
print(f"Original shape: {tensor.shape}")
reshaped = tensor.reshape(2, 3*4) # Combine the rows and columns into one
print(f"Shape of the reshaped tensor: {reshaped.shape}")
reshaped

Original shape: torch.Size([2, 3, 4])
Shape of the reshaped tensor: torch.Size([2, 12])


tensor([[6, 0, 0, 0, 0, 1, 3, 0, 1, 1, 7, 9],
        [4, 3, 8, 9, 3, 7, 8, 1, 4, 1, 6, 3]])

Hvis det er en shape-dimensjon vi ikke har styr på og ofte varierer kan vi la PyTorch regne den ut selv ved å spesifisere `-1` for **en** av posisjonene.

Dette er veldig nyttig når vi ønsker å samle flere datapunkt i en tensor. Vi vet gjerne hvordan et datapunkt ser ut, feks vil et bilde ha en bestemt oppløsning og antall fargekanaler. Disse verdiene vil være felles for alle datapunkt i datasettet. Denne prosessen kalles for batching, og vi kommer tilbake til det senere. 

In [72]:
reshaped = tensor.reshape(-1, 3*4)
print(reshaped)
print(f"PyTorch infers the first shape dimension to be {reshaped.shape[0]}")

tensor([[6, 0, 0, 0, 0, 1, 3, 0, 1, 1, 7, 9],
        [4, 3, 8, 9, 3, 7, 8, 1, 4, 1, 6, 3]])
PyTorch infers the first shape dimension to be 2


Når vi har flere _shape_-dimensjoner har vi muligheten til å spesifisere dimensjoner av interesse for noen av operasjonene.

In [73]:
torch.manual_seed(42)
tensor = torch.randint(low=0, high=10, size=(2, 3))
tensor

tensor([[2, 7, 6],
        [4, 6, 5]])

In [74]:
print(f"Summing across rows: {tensor.sum(dim=0)}")
print(f"Summing across columns: {tensor.sum(dim=1)}")

Summing across rows: tensor([ 6, 13, 11])
Summing across columns: tensor([15, 15])


In [75]:
print(f"Min values across rows: {tensor.min(dim=0).values}")
print(f"Min values across columns: {tensor.min(dim=1).values}")

Min values across rows: tensor([2, 6, 5])
Min values across columns: tensor([2, 4])


### Legge til og fjerne tomme dimensjoner
Antall shape-dimensjoner i to tensorer kan være forskjellige, mens dataene tilsynelatende har en lik struktur. 
For å illustrere dette starter vi med en vektor som inneholder 4 features. Denne legger vi til en _tom_ shape-dimensjon på med `.unsqueeze(dim=X)`

In [76]:
tensor = torch.randint(low=0, high=10, size=(4,))
print(f"Original tensor is {tensor} with shape {tensor.shape}")
modified = tensor.unsqueeze(0)
print(f"Modified tensor is {modified} with shape {modified.shape}")

Original tensor is tensor([0, 4, 0, 3]) with shape torch.Size([4])
Modified tensor is tensor([[0, 4, 0, 3]]) with shape torch.Size([1, 4])


Vi kan også fjerne tomme dimensjoner.

In [77]:
tensor = torch.randint(low=0, high=10, size=(4, 1)) # 4 rows and 1 column
tensor

tensor([[8],
        [4],
        [0],
        [4]])

In [78]:
modified = tensor.squeeze(1)
modified

tensor([8, 4, 0, 4])

PyTorch er litt streng, og vi kommer til å oppleve at den setter krav til hvilken shape tensorene våre skal ha når vi bruker hjelpemetodene i biblioteket. Nevrale nettverk som vi bygger med PyTorch (mer om det i neste del) forventer at input-dataen kommer som flere datapunkt samlet. Hvis vi bare skal sende inn et datapunkt må vi dytte inn en tom batch-dimensjon framst i tensoren. 

## Flytting av tensorer mellom enheter
Vi skriver PyTorch-kode gjennom Python, men mye av beregningene foregår i et C++ backend. Dette er nødvendig for at ressurskrevende operasjoner kan gjøres på grafikkort. Det eneste vi trenger å gjøre er å flytte tensorer dit. 

In [79]:
print(f"Default tensor location: {tensor.device}")

Default tensor location: cpu


In [80]:
print(f"GPU is available: {torch.cuda.is_available()}")

GPU is available: False


In [81]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

[CUDA](https://en.wikipedia.org/wiki/CUDA) er API-et som brukes for å kjøre beregninger på NVIDIA-grafikkort. 

In [82]:
tensor = tensor.to(device)
print(f"Tensor location: {tensor.device}")

Tensor location: cpu
